
<a href="https://colab.research.google.com/github/kokchun/Databehandling-AI22/blob/main/Exercises/E02_merging.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; to see hints and answers.

---
# Merging exercises

---
These are introductory exercises in Pandas with focus in **syntax, indexing, data selection, missing data, aggregations, visualizations**, **data cleaning**, **merging**, **concatenation**, **joining**, **parse html tables** .

<p class = "alert alert-info" role="alert"><b>Note</b> all datasets used in this exercise can be found under Data folder of the course Github repo</p>

<p class = "alert alert-info" role="alert"><b>Note</b> that there are usually many ways to achieve same results using operations in pandas. Feel free to investigate several ways. </p>

<p class = "alert alert-info" role="alert"><b>Note</b> that in cases when you start to repeat code, try not to. Create functions to reuse code instead. </p>

<p class = "alert alert-info" role="alert"><b>Remember</b> to use <b>descriptive variable, function, index </b> and <b> column names</b> in order to get readable code </p>

<p class = "alert alert-info" role="alert"><b>Remember</b> to format your input questions in a pedagogical way to guide the user

The number of stars (\*), (\*\*), (\*\*\*) denotes the difficulty level of the task

---

## 1. Swedish demographic data (*)

Go to Swedish-language wikipedia page [Sveriges demografi](https://sv.wikipedia.org/wiki/Sveriges_demografi). 

&nbsp; a) Read in the table under "Befolkningsstatistik sedan 1900" into a DataFrame

&nbsp; b) Choose to do some EDA (exploratory data analysis) on this dataset. And draw some relevant graphs.

&nbsp; c) Now we want to go backwards in time (before 1900) to see how population has changed in Sweden. Read in the table under history and keep the data of "Folkmängd" from 1570-1865. 

| År   | Folkmängd |
| ---- | --------- | 
| 1570 | 900000    |     
| 1650 | 1225000   |
| 1700 | 1485000   |
| 1720 | 1350000   |
| 1755 | 1878000   |
| 1815 | 2465000   |
| 1865 | 4099000   |


&nbsp; d) Now concatenate this with the table from 1900 so that you have population data from 1570 to 2020. Note that you may need to clean the data in order for it to fit properly. Also you may be able to do this in several ways. 

&nbsp; e) Draw a graph of population data from 1570-2020.

&nbsp; f) Try to see if you can plot something from this data that might show the effects of the Covid-19 pandemic. It"s good to zoom in the data, i.e. plot a shorter time frame instead of all time frames. 

&nbsp; g) Try to see if you can plot something from this data that might capture the effects imigration and emigration waves.

&nbsp; h) Feel free to find other interesting facts about the Swedish demographics based on this data. 

<details>
<summary>Hint</summary>

Useful methods:
- append()
- join()
- concat()
- merge()

</details>

<br/>

<details>

<summary>Answer</summary>

![Sweden population data 1952-2020](../assets/sverige_befolkning_tid.png)

</details>

In [114]:
# 1.a)

import pandas as pd
pd.options.mode.copy_on_write = True        # recommended by pandas due to copy-on-write becoming standard in pandas 3
import numpy as np
from IPython.display import display
import plotly_express as px 
import matplotlib.pyplot as plt
import seaborn as sns

def scrape_table(url, table_number):
    df = pd.read_html(url)
    table = df[table_number]
    path = "../Data/" + url.split("/")[-1] + f"_{str(table_number)}.csv"
    table.to_csv(path)
    return path


def read_table(path):
    table = pd.read_csv(path)
    return table

# loc = scrape_table("https://sv.wikipedia.org/wiki/Sveriges_demografi", 6) # sparad som CSV
swe_raw = read_table("/home/albot/coding/repos/databehandling-albot/Data/Sveriges_demografi_6.csv") # tidigare arg=loc
swe = swe_raw.rename(columns={"Unnamed: 0": "År", 
                              "Medel- folkmängd": "Folkmängd",
                              "Naturlig befolknings- förändring": "Naturlig_befolkningsförändring",
                              "Nativitet (per 1000)": "Nativitet_per_1000", 
                              "Dödstal (per 1000)": "Dödstal_per_1000",
                              "Naturlig befolknings- förändring (per 1000)": "Naturlig_befolkningsförändring_per_1000",
                              "Total fertilitet": "Total_fertilitet"})
swe = swe.drop("Unnamed: 0.1", axis=1).set_index("År")
swe = swe.dropna(how="any")
for col in swe.columns[:4]:
    swe[col] = swe[col].astype(str).str.strip().str.replace(",", ".").str.replace(" ", "").astype(float)
# display(swe)

In [115]:
# 1.b)

def plot_pop_time(df):
    fig = px.line(df, x=df.index, y="Folkmängd", title="Sveriges befolkning över tid")
    fig.show()


def plot_born_dead():
    swe_born_dead = swe[["Födda", "Döda", "Naturlig_befolkningsförändring"]]
    swe_born_dead = swe_born_dead.reset_index()
    swe_bd_melted = swe_born_dead.melt(id_vars="År", value_vars=["Födda", "Döda", "Naturlig_befolkningsförändring"], 
                                    var_name="Category", value_name="Count")
    fig = px.line(swe_bd_melted, x="År", y="Count", color="Category", 
                  title="Born and Dead Over Time")
    fig.show()


def plot_per_1000():
    swe_per_1000 = swe[["Nativitet_per_1000", "Dödstal_per_1000", "Naturlig_befolkningsförändring_per_1000", "Total_fertilitet"]]
    swe_per_1000 = swe_per_1000.reset_index()
    swe_p1_melted = swe_per_1000.melt(id_vars="År", value_vars=["Nativitet_per_1000", "Dödstal_per_1000", "Naturlig_befolkningsförändring_per_1000", "Total_fertilitet"], 
                                    var_name="Category", value_name="Count")
    fig = px.line(swe_p1_melted, x="År", y="Count", color="Category", 
                  title="Population per 1000")
    fig.show()


# plot_pop_time()
# plot_born_dead()
# plot_per_1000()

In [116]:
# 1.c)

# loc = scrape_table("https://sv.wikipedia.org/wiki/Sveriges_demografi", 1) # sparad som CSV
swe_his_raw = read_table("/home/albot/coding/repos/databehandling-albot/Data/Sveriges_demografi_1.csv") # tidigare arg=loc
swe_his = swe_his_raw.rename(columns={"Vid utgången av år": "År",
                              "Folkmängd": "Folkmängd", 
                              "Årlig tillväxt": "Årlig_tillväxt_totalt",
                              "Årlig tillväxt.1": "Årlig_tillväxt_promille"})
swe_his = swe_his.drop(["Unnamed: 0", "Årlig_tillväxt_totalt", "Årlig_tillväxt_promille"], axis=1).set_index("År")
swe_his = swe_his.iloc[1:-4, :]
swe_his["Folkmängd"] = swe_his["Folkmängd"].astype(str).str.strip().str.replace("\xa0", "").str.replace(" ", "").astype(float)  # replacing "\xa0" due to prior ValueError: could not convert string to float: '900\xa0000'

# display(swe_his)

In [117]:
# 1.d)

swe_pop = swe["Folkmängd"]
swe_pop = pd.concat([swe_his, swe_pop])
# display(swe_pop.tail(100))

In [118]:
# 1.e)

# plot_pop_time(swe_pop)

In [126]:
# 1.f)

def plot_largest_death_increases():
    swe_death = swe[["Döda"]]
    swe_death["Diff_%"] = [np.nan] + [(np.divide((swe_death.iloc[i, 0] - swe_death.iloc[i-1, 0]), (swe_death.iloc[i, 0] + swe_death.iloc[i-1, 0])) * 100) for i in range(1, len(swe_death))]
    swe_death_top10 = swe_death.sort_values(by="Diff_%", ascending=False).head(10)
    swe_death_top10 = swe_death_top10.sort_index()
    ave = float(swe_death["Diff_%"].mean())
    mid = float(swe_death["Diff_%"].median())
    plt.figure(figsize=(10, 8))
    ax = sns.barplot(x=swe_death_top10.index, y=swe_death_top10["Diff_%"], zorder=2)
    ax.axhline(ave, color="red", linestyle="--", label="Genomsnitt")
    for container in ax.containers:
        ax.bar_label(container, fmt="%.2f", label_type="edge", padding=-15, color="white", fontweight="bold")
    plt.yticks(np.arange(0, 16, 1))
    plt.title("År med störst procentuell ökning av döda 1900-2020\n", fontweight="bold")
    plt.ylabel("Procentuell ökning\n", fontweight="bold")
    plt.xlabel("\nÅrtal", fontweight="bold")
    plt.grid(axis="y", zorder=1)
    plt.legend()
    plt.show()
    display(swe_death_top10)


# plot_largest_death_increases()

In [120]:
# 1.g)

def plot_migration():       # NOTE: antalet döda kan inkludera immigranter vilket påverkar hela uträkningen
    swe_mig = swe[["Folkmängd", "Födda", "Döda"]]
    swe_mig["Differens"] = swe_mig["Folkmängd"].diff()
    swe_mig["Migration"] = swe_mig["Folkmängd"].diff() - (swe_mig["Födda"] - swe_mig["Döda"])
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=swe_mig, x=swe_mig.index, y="Differens", label="Befolkningsförändring", linestyle="solid")
    sns.lineplot(data=swe_mig, x=swe_mig.index, y="Migration", label="Migration", linestyle="dashed")
    sns.lineplot(data=swe_mig, x=swe_mig.index, y="Födda", label="Födda", linestyle="dashdot")
    sns.lineplot(data=swe_mig, x=swe_mig.index, y="Döda", label="Döda", linestyle="dotted")
    plt.axhline(0, color="black")
    plt.title("Korrelation mellan befolkningsförändring och migration\n", fontweight="bold")
    plt.xlabel("\nÅrtal", fontweight="bold")
    plt.ylabel("Antal\n", fontweight="bold")
    plt.yticks(np.arange(-30000, 150000, 10000))
    plt.grid(axis="y")
    plt.legend()
    plt.show()

# plot_migration()

In [121]:
# 1.h)
# Feel free to find other interesting facts about the Swedish demographics based on this data.

---
## 2. Denmark demographic data (*)

Go to the Danish-language wikipedia page [Danmarks demografi](https://da.wikipedia.org/wiki/Danmarks_demografi). 

&nbsp; a) Read in the table under "Demografiske data" into a DataFrame (*)

&nbsp; b) Clean the data and draw a graph of population against year from 1769-2020. (**)


In [131]:
def clear_data(df):
    df = df.dropna(how="any")
    df.columns = ["År", "Folkmängd"]
    df["År"] = df["År"].astype(int)
    df["Folkmängd"] = df["Folkmängd"].astype(str).str.strip().str.replace(".", "").str.replace("[7]", "").astype(float)
    df = df.set_index("År")
    return df

# loc = scrape_table("https://da.wikipedia.org/wiki/Danmarks_demografi", 0) # sparad som CSV
den_raw = read_table("/home/albot/coding/repos/databehandling-albot/Data/Danmarks_demografi_0.csv") # tidigare arg=loc
den = den_raw.drop("Unnamed: 0", axis=1)
den1 = den.iloc[0:, 0:2]
den2 = den.iloc[0:, 2:4]
den1 = clear_data(den1)
den2 = clear_data(den2)
den = pd.concat([den1, den2])
# display(den)

In [123]:
def plot_den():
    den_modern = den.iloc[2:, :]
    fig = px.line(data_frame=den_modern, x=den_modern.index, y="Folkmängd", title="Danmarks befolkning över tid")
    fig.show()


# plot_den()

---
## 3. Norwegian demographic data (*)

Go to Swedish-language wikipedia page [Norges demografi](https://sv.wikipedia.org/wiki/Norges_demografi). 

&nbsp; a) Read in the table under "Befolkningsstatistik sedan 1900" into a DataFrame

&nbsp; b) You see some missing data in column "Total fertilitet". Go to the [English page](https://en.wikipedia.org/wiki/Demographics_of_Norway) and read in the data from "Vital statistics since 1900".  

&nbsp; c) Pick out the fertility column from b) dataset, merge it into a) dataset and clean the data so that you only have columns "År", "Folkmängd", "Fertilitet". 


In [132]:
loc = scrape_table("https://sv.wikipedia.org/wiki/Norges_demografi", 2)
loc

'../Data/Norges_demografi_2.csv'

---
## 4. Merge Sweden-Norway (*)

Create a population graph and a fertility graph showing Sweden and Norway.

<details>

<summary>Answer</summary>

![Fertilitet Norge och Sverige](../assets/fertilitet_sv_no.png)

![Folkmängd Norge och Sverige](../assets/folkmangd_sverige_norge.png)

</details>

In [127]:
# Merge Sweden - Denmark

swe_1900 = swe_pop.iloc[8:, :]
swe_1900["Land"] = "Sverige"
swe_1900 = swe_1900.reset_index()
den_1900 = den.iloc[14:, :]
den_1900["Land"] = "Danmark"
den_1900 = den_1900.reset_index()
sweden = pd.concat([swe_1900, den_1900], ignore_index=True)
sweden = sweden.sort_values("År")

In [125]:
fig= px.line(data_frame=sweden, x="År", y="Folkmängd", color="Land", title="Sveriges och Danmarks befolkning över tid")
fig.show()

---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---